In [1]:
#Importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
#loading the data files into pandas dataframe
gross = pd.read_csv('data/bom.movie_gross.csv')
imdb_name = pd.read_csv('data/imdb.name.basics.csv')
imdb_title_akas = pd.read_csv('data/imdb.title.akas.csv')
imdb_title_basics = pd.read_csv('data/imdb.title.basics.csv')
imdb_title_crew = pd.read_csv('data/imdb.title.crew.csv')
imdb_title_principals = pd.read_csv('data/imdb.title.principals.csv')
imdb_title_ratings = pd.read_csv('data/imdb.title.ratings.csv')
rt_movie_info = pd.read_csv('data/rt.movie_info.tsv', delimiter = '\t')
rt_reviews = pd.read_csv('data/rt.reviews.tsv', delimiter = '\t', encoding = 'unicode_escape')
tmdb_movies = pd.read_csv('data/tmdb.movies.csv')
tn_movie_budgets = pd.read_csv('data/tn.movie_budgets.csv') #No Missing values


In [3]:
# setting and renaming index of tmdb_movies dataset
tmdb_movies.set_index('Unnamed: 0',inplace = True)
tmdb_movies.index.rename('index',inplace = True)


In [4]:
#Filled missing values with of gross foriegn value to 0 to convert the type of the column to float
gross.foreign_gross = gross.foreign_gross.fillna(0)



In [5]:
#This function takes a dataframe and column name as arguments
#It removes '$' and ',' from the column values and convert the column type to float
#This function also divides the number with a million to make the numbers easy to read
def string_to_float(df,column):
    df[column] =  df[column].replace({'\$':''}, regex = True)
    df[column] =  df[column].replace({',':''}, regex = True)
    df[column] =  round((df[column].astype(float))/1000000, 2)

# Calling the function on all columns from all datasets to get desired result
string_to_float(tn_movie_budgets,'production_budget')
string_to_float(tn_movie_budgets,'domestic_gross')
string_to_float(tn_movie_budgets,'worldwide_gross')
string_to_float(gross,'foreign_gross')


In [6]:
#since domestic_gross was already a float dividing it by a million to have same unit of gross throughout the project
gross.domestic_gross = gross.domestic_gross/1000000


In [7]:
#Creating a new column profit for profit/loss of movies
tn_movie_budgets['profit'] =  tn_movie_budgets['worldwide_gross'] - tn_movie_budgets['production_budget']




In [8]:
# top ten movies according to profit
#top_grossing_movies = list(tn_movie_budgets.sort_values(by = 'profit', ascending = True)['movie'])
# find movies that are profitable and not profitable
#tn_movie_budgets.sort_values(by = 'profit', ascending = False)['profit'].hist()
#sequels how good are sequels
#
tn_movie_budgets.sort_values(by = 'profit', ascending = False).head(10)
profitable_movies = list(tn_movie_budgets.sort_values(by = 'profit', ascending = False)['movie'].head(50))
not_profitable_movies = list(tn_movie_budgets.sort_values(by = 'profit')['movie'].head(50))



In [34]:
# Two datasets imdb_title_basics and tn_movie_budgets are merged to get profit and genres together in one table
profit_genre = pd.merge(imdb_title_basics[['tconst', 'start_year', 'runtime_minutes', 'genres','primary_title']], tn_movie_budgets[['release_date', 'movie','profit']], left_on = 'primary_title', right_on = 'movie', how='inner')
profit_genre.set_index('tconst', inplace = True)
profit_genre  = merged_set.drop_duplicates(subset = 'movie')

In [35]:
profit_genre.sort_values(by = 'profit', ascending = False).head()


,start_year,runtime_minutes,genres,primary_title,release_date,movie,profit
tconst,,,,,,,
tt1775309,2011,93.0,Horror,Avatar,"Dec 18, 2009",Avatar,2351.35
tt2495766,2012,NaN,Adventure,Titanic,"Dec 19, 1997",Titanic,2008.21
tt4154756,2018,149.0,"Action,Adventure,Sci-Fi",Avengers: Infinity War,"Apr 27, 2018",Avengers: Infinity War,1748.13
tt0369610,2015,124.0,"Action,Adventure,Sci-Fi",Jurassic World,"Jun 12, 2015",Jurassic World,1433.85
tt2820852,2015,137.0,"Action,Crime,Thriller",Furious 7,"Apr 3, 2015",Furious 7,1328.72


In [26]:
#Merging gross and tn_movie_budgets to get profit and studios in same table
profit_studio = pd.merge(gross[['title', 'studio', 'year']], tn_movie_budgets[['release_date', 'movie', 'profit']], left_on = 'title', right_on = 'movie', how='inner')


In [27]:
profit_studio.sort_values(by = 'profit', ascending = False).head()


,title,studio,year,release_date,movie,profit
1154,Avengers: Infinity War,BV,2018,"Apr 27, 2018",Avengers: Infinity War,1748.13
764,Jurassic World,Uni.,2015,"Jun 12, 2015",Jurassic World,1433.85
765,Furious 7,Uni.,2015,"Apr 3, 2015",Furious 7,1328.72
1155,Black Panther,BV,2018,"Feb 16, 2018",Black Panther,1148.26
1156,Jurassic World: Fallen Kingdom,Uni.,2018,"Jun 22, 2018",Jurassic World: Fallen Kingdom,1135.77


In [43]:

profit_crew = pd.merge(profit_genre[['movie','profit']], imdb_title_crew, on = 'tconst', how='inner')



In [45]:
profit_crew.sort_values(by = 'profit', ascending = False).head()


,tconst,movie,profit,directors,writers
950,tt1775309,Avatar,2351.35,nm3786927,"nm2179863,nm4392664"
1493,tt2495766,Titanic,2008.21,"nm4430776,nm5410036",nm5410036
1924,tt4154756,Avengers: Infinity War,1748.13,"nm0751577,nm0751648","nm1321655,nm1321656,nm0498278,nm0456158,nm0800..."
6,tt0369610,Jurassic World,1433.85,nm1119880,"nm0415425,nm0798646,nm1119880,nm2081046,nm0000341"
1602,tt2820852,Furious 7,1328.72,nm1490123,"nm0604555,nm0860155"


In [49]:
imdb_title_principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"
